In [1]:
# login
import sys
import os

project_root = os.path.dirname(os.path.dirname(os.getcwd()))
if project_root not in sys.path:
    sys.path.append(project_root)

from utils.brainLogin import get_session

session = get_session()

response = session.post("https://api.worldquantbrain.com/authentication")
print(response.status_code)
print(response.json())

201
{'user': {'id': 'GY27086'}, 'token': {'expiry': 14400.0}, 'permissions': ['REFERRAL']}


In [2]:
# get datafields
from utils.brainGetDataFields import get_datafields
from utils.brainDatafieldsSearchScopeConfig import get_search_scope 

search_scope = get_search_scope()
datafields = get_datafields(session, search_scope, dataset_id='fundamental6', search='', field_type='MATRIX')
datafields_list_fnd6 = datafields['id'].values
print(datafields_list_fnd6)



['assets' 'assets_curr' 'bookvalue_ps' 'capex' 'cash' 'cash_st' 'cashflow'
 'cashflow_dividends' 'cashflow_fin' 'cashflow_invst' 'cashflow_op' 'cogs'
 'current_ratio' 'debt' 'debt_lt' 'debt_st' 'depre_amort' 'ebit' 'ebitda'
 'employee' 'enterprise_value' 'eps' 'equity' 'fnd6_acdo' 'fnd6_acodo'
 'fnd6_acox' 'fnd6_acqgdwl' 'fnd6_acqintan' 'fnd6_adesinda_curcd'
 'fnd6_aldo' 'fnd6_am' 'fnd6_aodo' 'fnd6_aox' 'fnd6_aqc' 'fnd6_aqi'
 'fnd6_aqs' 'fnd6_beta' 'fnd6_capxv' 'fnd6_ceql' 'fnd6_ch' 'fnd6_ci'
 'fnd6_cibegni' 'fnd6_cicurr' 'fnd6_cidergl' 'fnd6_cik' 'fnd6_cimii'
 'fnd6_ciother' 'fnd6_cipen' 'fnd6_cisecgl' 'fnd6_citotal' 'fnd6_city'
 'fnd6_cld2' 'fnd6_cld3' 'fnd6_cld4' 'fnd6_cld5' 'fnd6_cptmfmq_actq'
 'fnd6_cptmfmq_atq' 'fnd6_cptmfmq_ceqq' 'fnd6_cptmfmq_dlttq'
 'fnd6_cptmfmq_dpq' 'fnd6_cptmfmq_lctq' 'fnd6_cptmfmq_oibdpq'
 'fnd6_cptmfmq_opepsq' 'fnd6_cptmfmq_saleq' 'fnd6_cptnewqv1300_actq'
 'fnd6_cptnewqv1300_apq' 'fnd6_cptnewqv1300_atq' 'fnd6_cptnewqv1300_ceqq'
 'fnd6_cptnewqv1300_dlttq' 

In [3]:
# template <group_compare_op>(<ts_compare_op>(<company_fundamentals>, <days>), <group>)

In [4]:
# 将datafield和operator替换到Alpha模板（框架）中 => group_rank(ts_rank({fundamental model data}, 252), industry), 批量生成Alpha
# 定义操作符列表
group_compare_op = ['group_rank', 'group_zscore', 'group_neutralize']
ts_compare_op = ['ts_rank', 'ts_zscore', 'ts_av_diff'] # 之前获取的fundamental6数据字段列表
company_fundamentals = datafields_list_fnd6 
days = [600, 200]
group = ['market', 'industry', 'subindustry', 'sector', 'densify(pv13_h_f1_sector)'] # 分组方式

# 初始化alpha表达式列表
alpha_expressions = []

# 嵌套循环生成所有可能的组合
for gco in group_compare_op:
    for tco in ts_compare_op:
        for cf in company_fundamentals:
            for d in days:
                for grp in group:
                    # 生成alpha表达式并添加到列表
                    alpha_expressions.append(f"{gco}({tco}({cf}, {d}), {grp})")

# 打印生成的表达式数量
print(f'there are total {len(alpha_expressions)} alpha expressions')

there are total 51660 alpha expressions


In [5]:
# 生成alpha列表
from utils.brainSimulationConfig import get_simulation_data

alpha_list = []
for alpha_expression in alpha_expressions:
    simulation_data = get_simulation_data(alpha_expression)
    alpha_list.append(simulation_data)

In [6]:
print(alpha_list.__len__())
print(alpha_list[0])

51660
{'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'USA', 'universe': 'TOP3000', 'delay': 1, 'decay': 0, 'neutralization': 'SUBINDUSTRY', 'truncation': 0.01, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'ON', 'language': 'FASTEXPR', 'visualization': False}, 'regular': 'group_rank(ts_rank(assets, 600), market)'}


In [ ]:
from utils.brainSimulation import run_alpha_simulation, rerun_alphas


In [7]:
from utils.brainSimulationRecord import check_progress

check_progress('20250113_181626')


批次 20250113_181626 的统计信息:
总任务数: 10000
已成功: 5354
已失败: 4646
待处理: 0
成功率: 53.54%
